# Implementación de una red AC-GAN con Keras<a id="top"></a>

<i><small>Autor: Algigantix<br>Última actualización: 2023-04-28</small></i></div>

## Introducción

Las redes generativas adversariales (GAN, de _Generative Adversarial Networks_) fueron presentadas por primera vez por Goodfellow et al. en el artículo [Generative Adversarial Networks](https://arxiv.org/abs/1406.2661) publicado en 2014. Este tipo de redes pueden ser utilizadas para la generación sintética de datos prácticamente idénticos a los originales.

Para la generación de estos datos se usan dos redes neuronales durante el entrenamiento: la **generadora**, que acepta un vector de entrada de ruido generado aleatoriamente y produce los datos de salida de aspecto similar a los datos auténticos, y la **discriminadora**, que intenta determinar si los datos que se le presentan son auténticos o generados.

Entrenando estas redes al mismo tiempo, una retroalimentando a la otra, dispondremos de un medio para generar datos prácticamente indistinguibles de los originales, o visto de otro modo, dispondremos de un medio para determinar si unos determinados datos son verdaderos o _fake_.

Una AC-GAN (_Auxiliary Classifier GAN_) es una variante en la que se incluye la clasificación por clases en la entrada de las dos redes, y la red discriminadora puede clasificar según probabilidad de que una imagen sea de una clase u  otra. Además emplean redes de convolución.

En nuestro caso, diseñaremos una red ACGAN que trate de falsificar billetes de euro.

## Imports y configuración

Primero importamos las bibliotecas que necesitamos

In [10]:
import collections

import enum

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

También incluimos un pip por si faltan algunos paquetes en la instalación

In [12]:
!pip install pydot

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Asimismo algunas configuraciones gráficas.

In [4]:
%matplotlib notebook
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (15, 8),'figure.dpi': 64})

## El dataset que usaremos

Utilizaremos un dataset propio de billetes de 5€, 10€, 20€ y 50€, con una dimensión de imágenes predeterminada por nosotros y de 3 canales.

In [3]:
WIDTH, HEIGHT, CHANNELS = 640, 480, 3

In [5]:
# Cargar imágenes TO-DO clasificar chicos, id buscando como cargar imágenes
# TO-DO BUSCAD COMO CARGAR IMAGENES NUESTRAS

(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data() #load_data()
dataset = np.concatenate((x_train, x_test))

print(f'Real images: {dataset.shape}')


# plot raw pixel data

# plot images from the training dataset
for i in range(100):
 # define subplot
 plt.subplot(10, 10, 1 + i)
 # turn off axis
 plt.axis('off')
 # plot raw pixel data
 plt.imshow(dataset[i])
plt.show()

Real images: (70000, 28, 28)


NameError: name 'load_data' is not defined

También definiremos las clases de billetes, pero indicando que solo usaremos 4

In [19]:
class Billetes(enum.Enum):
    """Cada una de los posibles billetes a falsificar."""
    los5euros = 0
    los10euros = 1
    los20euros = 2
    los50euros = 3
    los100euros = 4
    los200euros = 5
    los500euros = 6

In [18]:
num_billetes = 4

## Red generadora

La parte generadora de un modelo GAN se encarga de generar datos nuevos a partir de ruido aleatorio. Es de esperar que, a la larga, sea capaz de generar nuevos datos con una distribución similar a la de los elementos reales.

Y ya que hemos dicho _ruido aleatorio_, vamos con la definición de la longitud del vector de valores de entrada de ruido.

In [23]:
test = np.zeros(num_billetes)
print(test)
print(Billetes.los20euros.value)
test[Billetes.los20euros.value] = 1

print(test)

[0. 0. 0. 0.]
2
[0. 0. 1. 0.]


In [7]:
HIDDEN_FEATURES = 128

In [ ]:
def create_generator(hidden_features, output_shape, clase):
    
    in_label = tf.keras.layers.Input(shape=(1,))
    np.zeroes(n in Billetes())
    clasecilla =  np.zeros(num_billetes)
    clasecilla[clase.value] = 1
    
    # embedding for categorical input
    li = tf.keras.layers.Embedding(num_billetes, 50)(in_label)
    # linear multiplication
    n_nodes = 7 * 7
    li = tf.keras.layers.Dense(n_nodes, input_shape=(1,))
    # reshape to additional channel
    li = tf.keras.layers.Reshape((7, 7, 1))(li)
    
    n_nodes = 7 * 7
    li = Dense(n_nodes, kernel_initializer=init)(li)
    # reshape to additional channel
    li = Reshape((7, 7, 1))(li)
    
    in_label = tf.keras.layers.Input(shape=(hidden_features,))
    
     # foundation for 7x7 image
 n_nodes = 384 * 7 * 7
 gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
 gen = Activation('relu')(gen)
 gen = Reshape((7, 7, 384))(gen)
 # merge image gen and label input
 merge = Concatenate()([gen, li])
    
    return tf.keras.models.Sequential([
        
        tf.keras.layers.Input(shape=(hidden_features,)),
         #in_image = Input(shape=(WIDTH, HEIGHT, CHANNELS))
        tf.keras.layers.Conv2D(32, (3, 3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.2), padding='same', input_shape=(WIDTH, HEIGHT, CHANNELS))(x)
        tf.keras.layers.Dropout(0.5)
        
        tf.keras.layers.Conv2DTranspose(192, (5,5, 3), strides=(2,2), padding='same', kernel_initializer=init)
        
        
        tf.keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
        tf.keras.layers.BatchNormalization(momentum=0.8),
        tf.keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
        tf.keras.layers.BatchNormalization(momentum=0.8),
        tf.keras.layers.Dense(WIDTH*HEIGHT*CHANNELS, activation='sigmoid'),
        tf.keras.layers.Reshape(output_shape)
    ], name='Generator')

In [ ]:
 # weight initialization
 init = RandomNormal(stddev=0.02)
 # label input
 in_label = Input(shape=(1,))
 # embedding for categorical input
 li = tf.keras.layers.Embedding(n_classes, 50)(in_label)
 # linear multiplication
 n_nodes = 7 * 7
 li = tf.keras.layers.Dense(n_nodes, kernel_initializer=init)(li)
 # reshape to additional channel
 li = tf.keras.layers.Reshape((7, 7, 1))(li)
 # image generator input
 in_lat = Input(shape=(latent_dim,))
 # foundation for 7x7 image
 n_nodes = 384 * 7 * 7
 gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
 gen = Activation('relu')(gen)
 gen = Reshape((7, 7, 384))(gen)
 # merge image gen and label input
 merge = Concatenate()([gen, li])
 # upsample to 14x14
 gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
 gen = BatchNormalization()(gen)
 gen = Activation('relu')(gen)
 # upsample to 28x28
 gen = Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
 out_layer = Activation('tanh')(gen)
 # define model
 model = Model([in_lat, in_label], out_layer, name='Generator')
 return model

In [ ]:
generator = create_generator(HIDDEN_FEATURES, (WIDTH, HEIGHT, CHANNELS))
tf.keras.utils.plot_model(generator, show_shapes=True, show_layer_names=True)

## Red discriminadora

In [ ]:
def create_discriminator(input_shape,):
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
        tf.keras.layers.Dropout(0.9),
        tf.keras.layers.Dense(1, activation='sigmoid'),
        tf.keras.layers.Dropout(0.5),
    ], name='Discriminator')

In [ ]:
discriminator = create_discriminator(input_shape=(WIDTH, HEIGHT, CHANNELS))
tf.keras.utils.plot_model(discriminator, show_shapes=True, show_layer_names=False)